PREPROCESSING AND STACKING 

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import SimpleITK as sitk # Loading, saving, and converting medical images
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, ConcatItemsd,
    NormalizeIntensityd, Spacingd, CropForegroundd,
    Resized, Compose, SaveImaged
)
from monai.data import Dataset
import torch
from monai.transforms import Resize
from pathlib import Path

Preprocessing Training Data

In [ ]:
# Configuration
brats_root = Path(r"C:/Users/user/NeuroSegNet/data/raw/BraTS2025-GLI-PRE-Challenge-TrainingData/BraTS2025-GLI-PRE-Challenge-TrainingData")
output_dir = Path("data/processed_stacked/")
output_dir.mkdir(parents=True, exist_ok=True)

modalities = ["t1n", "t1c", "t2w", "t2f"]
print(f"Path exists: {brats_root.exists()}")
print(f"Sample folders: {[p.name for p in brats_root.glob('*')][:5]}")

In [ ]:
# Build MONAI transform pipeline
modality_keys = [f"image-{mod}" for mod in modalities]

preprocessing = Compose([
    LoadImaged(keys=modality_keys),
    EnsureChannelFirstd(keys=modality_keys),
    ConcatItemsd(keys=modality_keys, name="image", dim=0),  # Combine into (4, H, W, D)
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    Spacingd(keys="image", pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
    CropForegroundd(keys="image", source_key="image"),
    Resized(keys="image", spatial_size=(128, 128, 128)),
])

In [ ]:
# Robust Preprocessing Loop with Debug Prints
from monai.transforms import Resize

processed = 0
patients = sorted(list(brats_root.glob("BraTS-GLI-*")))

for patient_path in tqdm(patients, desc="Preprocessing & Stacking Patients"):
    patient_id = patient_path.name.strip()
    data = {}
    missing = False

    for mod in modalities:
        expected_file = patient_path / f"{patient_id}-{mod}.nii.gz"
        if not expected_file.exists():
            print(f"[Missing] {expected_file.name} in {patient_path}")
            missing = True
            break
        data[f"image-{mod}"] = str(expected_file)

    seg_path = patient_path / f"{patient_id}-seg.nii.gz"
    if not seg_path.exists():
        print(f"[Missing Segmentation] {seg_path}")
        missing = True

    if missing:
        continue

    dataset = Dataset(data=[data], transform=preprocessing)

    try:
        img_tensor = dataset[0]["image"]  # [4, H, W, D]
        img_tensor = img_tensor.unsqueeze(0).float()  # [1, 4, H, W, D]

        # Load segmentation
        seg = nib.load(str(seg_path))
        seg_data = seg.get_fdata()
        seg_tensor = torch.tensor(seg_data).unsqueeze(0).unsqueeze(0).float()  # [1, 1, H, W, D]

        # Resize seg if needed
        if seg_tensor.shape[2:] != img_tensor.shape[2:]:
            resize = Resize(spatial_size=img_tensor.shape[2:], mode="nearest")
            seg_tensor = resize(seg_tensor.squeeze(0)).unsqueeze(0)

        # Stack into [5, H, W, D]
        stacked_tensor = torch.cat([img_tensor, seg_tensor], dim=1).squeeze(0)  # [5, H, W, D]
        stacked_data = np.moveaxis(stacked_tensor.numpy(), 0, -1)  # [H, W, D, 5]

        # Save only stacked file
        stacked_path = output_dir / f"{patient_id}_ground_truth.nii.gz"
        nib.save(nib.Nifti1Image(stacked_data, affine=np.eye(4)), str(stacked_path))
        processed += 1

    except Exception as e:
        print(f"[Error] {patient_id}: {e}")

print(f"Successfully processed & stacked {processed} patients. Saved only to {output_dir}")

In [ ]:
# Visualize a sample saved 4-channel volume
saved_files = sorted(output_dir.glob("*.nii.gz"))
print(f"Files saved: {[f.name for f in saved_files]}")
print(f"Total saved: {len(saved_files)}")

# Only proceed if something was saved
if saved_files:
    sample_file = saved_files[0]
    img = nib.load(sample_file).get_fdata()
    img = np.transpose(img, (3, 0, 1, 2))  # (C, H, W, D)
    print(f"Loaded {sample_file.name}, new shape after transpose: {img.shape}")

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i in range(4):
        axes[i].imshow(img[i, :, :, img.shape[3] // 2], cmap='gray')
        axes[i].set_title(modalities[i])
        axes[i].axis("off")
    plt.tight_layout()
    plt.show()
else:
    print("No preprocessed images found.")

In [ ]:
# Define the path to the stacked data 
stacked_dir = Path("../data/processed_stacked")

# Load stacked files
stacked_files = sorted(stacked_dir.glob("*.nii.gz"))
print(f"Stacked files found: {len(stacked_files)}")

if stacked_files:
    # Load one stacked volume: [H, W, D, 5]
    img = nib.load(stacked_files[0]).get_fdata()
    img = np.transpose(img, (3, 0, 1, 2))  # [5, H, W, D]
    
    # Extract segmentation channel
    seg_channel = img[4]
    print("Segmentation unique values:", np.unique(seg_channel))
    print("Segmentation min/max:", seg_channel.min(), seg_channel.max())

    # === Visualization ===
    fig, axes = plt.subplots(1, 5, figsize=(25, 5))
    mid_slice = img.shape[3] // 2  # Mid axial slice

    for i in range(5):
        slice_img = img[i, :, :, mid_slice]
        if i < 4:
            axes[i].imshow(slice_img, cmap="gray")
            axes[i].set_title(f"Modality {i + 1}", fontsize=12)
        else:
            axes[i].imshow(
                slice_img,
                cmap="gray",
                vmin=0,
                vmax=np.max(seg_channel),
                interpolation="nearest"
            )
            axes[i].set_title("Segmentation", fontsize=12)
        axes[i].axis("off")

    plt.suptitle(f"{stacked_files[0].name} - Mid-slice preview", fontsize=16)
    plt.tight_layout()
    plt.show()

else:
    print("No stacked volumes found in:", stacked_dir)

Preprocessing Validation Data

In [ ]:
# Configuration
val_root = Path(r"C:/Users/user/NeuroSegNet/data/raw/BraTS2025-GLI-PRE-Challenge-ValidationData/BraTS2025-GLI-PRE-Challenge-ValidationData")

output_dir = Path("data/val_preprocessed/")
output_dir.mkdir(parents=True, exist_ok=True)

modalities = ["t1n", "t1c", "t2w", "t2f"]
print(f"Path exists: {val_root.exists()}")
print(f"Sample folders: {[p.name for p in val_root.glob('*')][:5]}")

In [ ]:
# Define a new preprocessing pipeline for validation data
modality_keys = [f"image-{mod}" for mod in modalities]
preprocessing_val = Compose([
    LoadImaged(keys=modality_keys),
    EnsureChannelFirstd(keys=modality_keys),
    ConcatItemsd(keys=modality_keys, name="image", dim=0),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    Spacingd(keys="image", pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
    CropForegroundd(keys="image", source_key="image"),
    Resized(keys="image", spatial_size=(128, 128, 128)),
    SaveImaged(keys="image", output_dir=output_dir, output_postfix="norm", separate_folder=False)
])

In [ ]:
# Preprocessing Loop with Debug Prints
processed = 0
patients = sorted(list(val_root.glob("BraTS-GLI-*")))

for patient_path in tqdm(patients, desc="Preprocessing Val MRI"):
    patient_id = patient_path.name.strip()  # strip whitespace
    data = {}
    missing = False

    for mod in modalities:
        expected_file = patient_path / f"{patient_id}-{mod}.nii.gz"
        
        if not expected_file.exists():
            # Print all files in the folder if missing
            available = list(patient_path.glob("*.nii.gz"))
            print(f"[Missing] Expected: {expected_file.name} in {patient_path}")
            print(f"Available files: {[f.name for f in available]}")
            missing = True
            break
        else:
            data[f"image-{mod}"] = str(expected_file)

    if missing:
        continue

    dataset = Dataset(data=[data], transform=preprocessing_val)

    try:
        _ = dataset[0]
        processed += 1
    except Exception as e:
        print(f"[Error] {patient_id}: {e}")

print(f"Successfully processed {processed} 4-channelled val MRI.")

In [ ]:
# Save preprocessed 4-channel validation MRI modalities volume to disk
output_dir = Path("../data/val_preprocessed")
output_dir.mkdir(parents=True, exist_ok=True)

for patient_path in tqdm(patients, desc="Saving preprocessed volumes"):
    patient_id = patient_path.name.strip()
    data = {}
    missing = False

    for mod in modalities:
        expected_file = patient_path / f"{patient_id}-{mod}.nii.gz"
        if not expected_file.exists():
            missing = True
            break
        else:
            data[f"image-{mod}"] = str(expected_file)

    if missing:
        continue

    dataset = Dataset(data=[data], transform=preprocessing_val)

    try:
        img = dataset[0]["image"]
        img_np = img.numpy()  # shape (4, H, W, D)
        img_np = np.transpose(img_np, (1, 2, 3, 0))  # (H, W, D, 4)

        out_img = nib.Nifti1Image(img_np, affine=np.eye(4))
        nib.save(out_img, output_dir / f"{patient_id}_val_image.nii.gz")
        print(f"Saved {patient_id}_val_image.nii.gz")
    except Exception as e:
        print(f"Failed to save {patient_id}: {e}")

In [ ]:
# Visualize a sample saved multimodal mri preprocessed data
saved_files = sorted(output_dir.glob("*.nii.gz"))  # uses the same output_dir from cell 11
print(f"Files saved: {[f.name for f in saved_files[:5]]}")
print(f"Total saved: {len(saved_files)}")

if saved_files:
    sample_file = saved_files[0]
    img = nib.load(sample_file).get_fdata()
    img = np.transpose(img, (3, 0, 1, 2))  # (C, H, W, D)
    print(f"Loaded {sample_file.name}, new shape after transpose: {img.shape}")

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i in range(4):
        axes[i].imshow(img[i, :, :, img.shape[3] // 2], cmap='gray')
        axes[i].set_title(modalities[i])
        axes[i].axis("off")
    plt.tight_layout()
    plt.show()
else:
    print(f"No preprocessed images found in {output_dir}")
